<h1 style="text-align:center; color: darkred;">JOBSTREET CRAWLING</h1>
<h3 style="text-align:center;">Created by : Faridl Gifari Kertabudi (fargif991@gmail.com)</h3>
<hr style="border: 1px solid black;">

In [1]:
from datetime import datetime, timedelta
import glob
import re
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
import pandas as pd
import numpy as np
import time
from datetime import datetime
import dateparser
from urllib.parse import urlparse, parse_qs
import gspread
from gspread_dataframe import set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials

In [2]:
today = datetime.today()
today_string = today.strftime('%Y-%m-%d')

In [3]:
chromedriver_path = "C:/chromedriver-win64/chromedriver.exe"

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")

service = Service(chromedriver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)

In [18]:
web_link = "https://www.jobstreet.co.id/id/data-analyst-jobs?"

# Navigate to the webpage containing the element
driver.get(web_link)

# Refresh the webpage
driver.refresh()

# Wait for a few seconds to see the effect
time.sleep(2)

driver.maximize_window()

In [5]:
def jobid_getter(url):
    # Parse the URL
    parsed_url = urlparse(url)

    # Extract the query parameters
    query_params = parse_qs(parsed_url.query)

    # Get the currentJobId
    current_job_id = query_params.get('jobId', [None])[0]
    return current_job_id

In [19]:
job_db2 = pd.DataFrame(columns=[
    'company'
    , 'role'
    , 'location'
    , 'post_duration'
    , 'total_application'
    , 'job_description'
    , 'level'
    , 'employment_status'
    , 'job_function'
    , 'industries'
    , 'url'
    , 'job_id'
    , 'salary'
    , 'company_total_employees'
    , 'company_rating'
    , 'company_total_review'
])

In [20]:
button_next_clicked = 0
indexing = 0
next_available = 1

while next_available:
    
    try:
        list_jobs_box = driver.find_elements(By.XPATH, "//div[@class='y735df0 _21bfxf1']")[0]
        list_jobs_box2 = list_jobs_box.find_elements(By.XPATH, ".//a[@data-testid='job-card-title']")

        for job_ind, job_box_ind in enumerate(list_jobs_box2):

            retrial = 0
            while retrial<3:
                try:
                    job_box_ind.click()
                    time.sleep(0.8)
                    job_detail_box = driver.find_elements(By.XPATH, "//div[@class='y735df0 _1iz8dgs6y']")[0]
                    job_role = driver.find_elements(By.XPATH, "//h1[@data-automation='job-detail-title']")[0].text
                    break
                except:
                    retrial +=1
                    print(f'try again {retrial}')
                    time.sleep(1)
                    if retrial >= 3:
                        continue

            job_company = driver.find_elements(By.XPATH, "//div[@class='y735df0 _1iz8dgsgi _1iz8dgs5a _1iz8dgsg2 _1iz8dgsfi _1akoxc52e _1akoxc52n']")[0]
            job_company2 = job_company.find_elements(By.XPATH, ".//div[@class='y735df0 _1iz8dgsr _1iz8dgsem _1iz8dgsej _1iz8dgsba _1iz8dgsb7 _1iz8dgs4y _1iz8dgsfm']")[0]
            job_company_final = job_company2.find_elements(By.XPATH, ".//span")[0].text

            try:
                job_rating = job_company.find_elements(By.XPATH, ".//div[@class='y735df0 _1iz8dgsr _1iz8dgsem _1iz8dgsej _1iz8dgsba _1iz8dgsb7 _1iz8dgs4y _1iz8dgsfm']")[1]
                job_rating_final = job_rating.find_elements(By.XPATH, ".//span[@class='y735df0 _1dtcwsp3']")[0].text
                job_total_review = job_rating.find_elements(By.XPATH, ".//a[@data-automation='job-header-company-review-link']")[0].text
            except:
                job_rating_final = np.nan
                job_total_review = np.nan

            job_location = driver.find_elements(By.XPATH, "//span[@data-automation='job-detail-location']")[0].text
            job_industries = driver.find_elements(By.XPATH, "//span[@data-automation='job-detail-classifications']")[0].text
            job_employment_type = driver.find_elements(By.XPATH, "//span[@data-automation='job-detail-work-type']")[0].text

            job_duration = driver.find_elements(By.XPATH, ".//span[contains(text(), 'Posted')]")[0].text

            job_description = driver.find_elements(By.XPATH, "//div[@data-automation='jobAdDetails']")[0].text
            try:
                job_salary = driver.find_elements(By.XPATH, "//span[@data-automation='job-detail-salary']")[0].text
            except:
                job_salary = np.nan

            try:
                profile_box = driver.find_elements(By.XPATH, "//section[.//div[@class='y735df0 _1akoxc50 _1akoxc57' and .//a[@data-automation='company-profile-profile-link']]]")[0]
                total_employee = profile_box.find_elements(By.XPATH, ".//span[@class='y735df0 _1iz8dgs4y _1iz8dgsr']")[1].text

                if 'employ' not in total_employee:
                    total_employee = np.nan
            except:
                total_employee = np.nan

            job_url = driver.current_url
            job_id = jobid_getter(job_url)

            job_db2.loc[indexing, 'company'] = job_company_final
            job_db2.loc[indexing, 'role'] = job_role
            job_db2.loc[indexing, 'location'] = job_location
            job_db2.loc[indexing, 'post_duration'] = job_duration
            job_db2.loc[indexing, 'total_application'] = np.nan
            job_db2.loc[indexing, 'level'] = np.nan
            job_db2.loc[indexing, 'employment_status'] = job_employment_type
            job_db2.loc[indexing, 'job_function'] = np.nan
            job_db2.loc[indexing, 'industries'] = job_industries
            job_db2.loc[indexing, 'job_description'] = job_description
            job_db2.loc[indexing, 'url'] = job_url
            job_db2.loc[indexing, 'job_id'] = job_id
            job_db2.loc[indexing, 'salary'] = job_salary
            job_db2.loc[indexing, 'company_total_employees'] = total_employee
            job_db2.loc[indexing, 'company_rating'] = job_rating_final
            job_db2.loc[indexing, 'company_total_review'] = job_total_review

            print(f'Berhasil scrap job ke {indexing+1}')
            indexing += 1

        try:
            cek_next = driver.find_elements(By.XPATH, ".//span[@class = 'y735df0 _1iz8dgsda' and contains(text(), 'Selanjutnya')]")[0]
        except Exception as e:
            print(f"An error occurred: {e}")
            next_available = 0
        
        cek_next.click()
        time.sleep(1)
    
    except Exception as e:
        print(f"An error occurred: {e}")
        next_available = 0

Berhasil scrap job ke 1
Berhasil scrap job ke 2
Berhasil scrap job ke 3
Berhasil scrap job ke 4
Berhasil scrap job ke 5
Berhasil scrap job ke 6
Berhasil scrap job ke 7
Berhasil scrap job ke 8
Berhasil scrap job ke 9
Berhasil scrap job ke 10
Berhasil scrap job ke 11
Berhasil scrap job ke 12
Berhasil scrap job ke 13
Berhasil scrap job ke 14
Berhasil scrap job ke 15
Berhasil scrap job ke 16
Berhasil scrap job ke 17
Berhasil scrap job ke 18
Berhasil scrap job ke 19
Berhasil scrap job ke 20
Berhasil scrap job ke 21
Berhasil scrap job ke 22
Berhasil scrap job ke 23
Berhasil scrap job ke 24
Berhasil scrap job ke 25
Berhasil scrap job ke 26
Berhasil scrap job ke 27
Berhasil scrap job ke 28
Berhasil scrap job ke 29
Berhasil scrap job ke 30
Berhasil scrap job ke 31
Berhasil scrap job ke 32
Berhasil scrap job ke 33
Berhasil scrap job ke 34
Berhasil scrap job ke 35
Berhasil scrap job ke 36
Berhasil scrap job ke 37
Berhasil scrap job ke 38
Berhasil scrap job ke 39
Berhasil scrap job ke 40
Berhasil 

Berhasil scrap job ke 320
Berhasil scrap job ke 321
Berhasil scrap job ke 322
Berhasil scrap job ke 323
Berhasil scrap job ke 324
Berhasil scrap job ke 325
Berhasil scrap job ke 326
Berhasil scrap job ke 327
Berhasil scrap job ke 328
Berhasil scrap job ke 329
Berhasil scrap job ke 330
Berhasil scrap job ke 331
Berhasil scrap job ke 332
Berhasil scrap job ke 333
Berhasil scrap job ke 334
Berhasil scrap job ke 335
Berhasil scrap job ke 336
Berhasil scrap job ke 337
Berhasil scrap job ke 338
Berhasil scrap job ke 339
Berhasil scrap job ke 340
Berhasil scrap job ke 341
Berhasil scrap job ke 342
Berhasil scrap job ke 343
Berhasil scrap job ke 344
Berhasil scrap job ke 345
Berhasil scrap job ke 346
Berhasil scrap job ke 347
Berhasil scrap job ke 348
Berhasil scrap job ke 349
Berhasil scrap job ke 350
Berhasil scrap job ke 351
Berhasil scrap job ke 352
Berhasil scrap job ke 353
Berhasil scrap job ke 354
Berhasil scrap job ke 355
Berhasil scrap job ke 356
Berhasil scrap job ke 357
Berhasil scr

Berhasil scrap job ke 635
Berhasil scrap job ke 636
Berhasil scrap job ke 637
Berhasil scrap job ke 638
Berhasil scrap job ke 639
Berhasil scrap job ke 640
Berhasil scrap job ke 641
Berhasil scrap job ke 642
Berhasil scrap job ke 643
Berhasil scrap job ke 644
Berhasil scrap job ke 645
Berhasil scrap job ke 646
Berhasil scrap job ke 647
Berhasil scrap job ke 648
Berhasil scrap job ke 649
Berhasil scrap job ke 650
Berhasil scrap job ke 651
Berhasil scrap job ke 652
Berhasil scrap job ke 653
Berhasil scrap job ke 654
Berhasil scrap job ke 655
Berhasil scrap job ke 656
Berhasil scrap job ke 657
Berhasil scrap job ke 658
Berhasil scrap job ke 659
Berhasil scrap job ke 660
Berhasil scrap job ke 661
Berhasil scrap job ke 662
Berhasil scrap job ke 663
Berhasil scrap job ke 664
Berhasil scrap job ke 665
Berhasil scrap job ke 666
Berhasil scrap job ke 667
Berhasil scrap job ke 668
Berhasil scrap job ke 669
Berhasil scrap job ke 670
Berhasil scrap job ke 671
Berhasil scrap job ke 672
Berhasil scr

Berhasil scrap job ke 949
Berhasil scrap job ke 950
Berhasil scrap job ke 951
Berhasil scrap job ke 952
Berhasil scrap job ke 953
Berhasil scrap job ke 954
Berhasil scrap job ke 955
Berhasil scrap job ke 956
Berhasil scrap job ke 957
Berhasil scrap job ke 958
Berhasil scrap job ke 959
Berhasil scrap job ke 960
Berhasil scrap job ke 961
Berhasil scrap job ke 962
Berhasil scrap job ke 963
Berhasil scrap job ke 964
Berhasil scrap job ke 965
Berhasil scrap job ke 966
Berhasil scrap job ke 967
Berhasil scrap job ke 968
Berhasil scrap job ke 969
Berhasil scrap job ke 970
Berhasil scrap job ke 971
Berhasil scrap job ke 972
Berhasil scrap job ke 973
Berhasil scrap job ke 974
Berhasil scrap job ke 975
Berhasil scrap job ke 976
Berhasil scrap job ke 977
Berhasil scrap job ke 978
Berhasil scrap job ke 979
Berhasil scrap job ke 980
Berhasil scrap job ke 981
Berhasil scrap job ke 982
Berhasil scrap job ke 983
Berhasil scrap job ke 984
Berhasil scrap job ke 985
Berhasil scrap job ke 986
Berhasil scr

In [21]:
def convert_to_days(time_str):
    match = re.search(r'(\d+|\d+\+?) (menit|jam|hari)', time_str)
    if not match:
        return None
    value, unit = match.groups()
    value = int(value.replace('+', ''))
    if unit == 'jam':
        return value / 24
    elif unit == 'hari':
        return value
    elif unit == 'menit':
        return value / 1440
    return None

# Function to replace specific word
def replace_word(text, target_word, replacement_word):
    # Compile the pattern with case insensitivity
    pattern = re.compile(re.escape(target_word), re.IGNORECASE)
    # Replace the target word with the replacement word
    return pattern.sub(replacement_word, text)

def find_matches(text, pattern, watchlist):
    matches = pattern.findall(text)
    # Map matches to their original case in the watchlist
    original_matches = {match.upper(): match for match in watchlist}
    matched_words = [original_matches[match.upper()] for match in matches]
    
    # Remove duplicates and return a comma-separated string
    return ', '.join(sorted(set(matched_words), key=lambda x: text.upper().index(x.upper())))

def contains_keywords(text):
    # Convert text to uppercase for case-insensitive comparison
    text_upper = text.upper()
    
    # Check for exclusion keywords first
    if pattern_exp_exclusion.search(text_upper):
        return False
    
    # Check for English keyword combinations
    primary_match_en = pattern_exp_en.search(text_upper)
    year_match_en = year_pattern_en.search(text_upper)
    
    # Check for Indonesian keyword combinations
    primary_match_id = pattern_exp_id.search(text_upper)
    year_match_id = year_pattern_id.search(text_upper)
    
    # Return True if the correct combinations are found
    if primary_match_en and year_match_en:
        return True
    if primary_match_id and year_match_id:
        return True
    
    # Return False if only partial matches are found
    if primary_match_en and not year_match_en:
        return False
    if primary_match_id and not year_match_id:
        return False

    # Default to False if no conditions are met
    return False

In [22]:
job_db2['posted_since_days_ago'] = job_db2['post_duration'].apply(lambda x: convert_to_days(x))

In [24]:
job_db2['data_scraped_at'] = today_string

In [25]:
job_data_analyst = job_db2[
    (job_db2['role'].str.upper().str.contains('ANAL', na=False))
    | (job_db2['role'].str.upper().str.contains('BI DEV', na=False))
    | (job_db2['role'].str.upper().str.contains('INTELLIGENCE', na=False))
    | (job_db2['role'].str.upper().str.contains('INSIGHT', na=False))
    | (job_db2['role'].str.upper().str.contains('POWER BI', na=False))
].reset_index(drop=True)

job_data_analyst_final = job_data_analyst[
    (~job_data_analyst['role'].str.upper().str.contains('SOFTWARE', na=False)) 
    & (~job_data_analyst['role'].str.upper().str.contains('SECURITY', na=False))
    & (~job_data_analyst['role'].str.upper().str.contains('IT BUSINESS ANALYST', na=False))
    & (~job_data_analyst['role'].str.upper().str.contains('SYSTEM ANALYST', na=False))
].reset_index(drop=True)

In [26]:
list_tools = [
    'CANVA', 'CAMTASIA', 'SAP', 'EXCEL', 'VISIO', 'POWERBI', 'POWER BI', 'TABLEAU', 'LOOKER STUDIO', 'LOOKER', 
    'POWER POINT', 'VISUAL BASIC', 'MACRO', 'QLIKSENSE', 'PYTHON', 'BIGQUERY', 'BIG QUERY', 'AWS', 'AWS ATHENA', 
    'A/B TEST', ' R ', 'PHYTON', 'DBT', 'GITLAB', 'GITHUB', 'GOOGLE CLOUD', 'NOSQL', 'MYSQL', 'POSTGRESQL', 'SQL', 
    'G-SUITE', 'GOOGLE ADS', 'META ADS', 'SQL SERVER', '.NET', 'PPT', 'POWERPOINT', 'GOOGLE DATA STUDIO', 'AIRFLOW', 
    'HIVE', 'JUPYTER', 'HTML', 'CSS', 'JAVASCRIPT', 'JAVA', 'STATA', 'REDSHIFT', 'ORACLE', 'HADOOP', 'AZURE', 'SCALA', 
    'C++', 'CI/CD', 'SELENIUM', 'KATALON', 'SPSS', 'GCP', 'APACHE SPARK', 'SPARK', 'VBA', 'PENTAHO', 'MONGODB', 'DAX', 
    'HDFS', 'SSIS', 'SSAS', 'SSRS', 'TENSORFLOW', 'PYTORCH'
]

list_hardskills = [
    'BIG DATA', 'DATA CLEANSING', 'DATA WAREHOUSE', 'DATA VISUALIZATION', 'BI TOOLS', 'DATA AUTOMATION', 'DATA MINING', 
    'DATA MANAGEMENT', 'LMS', 'TECHNICAL GUIDANCE', 'BRD', 'FSD', 'TSD', 'UML', 'MS. OFFICE', 'CLUSTERING', 'REGRESSION', 
    'CLASSIFICATION', 'DATA MODEL', 'DATA MANIPULATION', 'DASHBOARD', 'ETL', 'DESCRIPTIVE', 'PREDICTIVE', 'DATA PROFILING', 
    'DATA PIPELINE', 'HLOOKUP', 'VLOOKUP', 'PIVOT', 'DATAMART', 'DATA MART', 'DATA EXTRACTION', 'MACHINE LEARNING', 
    'INFERENTIAL STATISTIC', 'COMPUTER VISION', 'NATURAL LANGUAGE PROCESSING', 'NLP', 'FINANCIAL REPORT', 'AB TEST', ' LLM ', 
    'SCRAPING', 'WEB CRAWL', 'OLAP', 'DATA BESAR', 'DATA COLLECTION', 'FRAUD'
]

list_language = [
    'ENGLISH', 'MANDARIN', 'BAHASA'
]

list_exp_en = ['YEARS IN', 'EXPERIENCE', 'EXP']
list_exp_id = ['PENGALAMAN']
year_keyword_en = ['YEAR']
year_keyword_id = ['TAHUN']
list_exp_exclusion = ['GRADUATE', 'STUDENT', 'LESS THAN 1 YEAR']

# Compile the regex pattern
pattern_tools = re.compile('|'.join([re.escape(word) for word in list_tools]), re.IGNORECASE)
pattern_hardskills = re.compile('|'.join([re.escape(word) for word in list_hardskills]), re.IGNORECASE)
pattern_exp_en = re.compile('|'.join(list_exp_en), re.IGNORECASE)
pattern_exp_id = re.compile('|'.join(list_exp_id), re.IGNORECASE)
year_pattern_en = re.compile('|'.join(year_keyword_en), re.IGNORECASE)
year_pattern_id = re.compile('|'.join(year_keyword_id), re.IGNORECASE)
pattern_exp_exclusion = re.compile('|'.join(list_exp_exclusion), re.IGNORECASE)
pattern_lang = re.compile('|'.join(list_language), re.IGNORECASE)

In [27]:
# TOOLS DETECTION
job_data_analyst_final['tools_mentioned'] = job_data_analyst_final['job_description'].apply(lambda x: find_matches(x, pattern_tools, list_tools))
job_data_analyst_final['tools_mentioned'] = job_data_analyst_final['tools_mentioned'].apply(lambda x: replace_word(x, 'PHYTON', 'PYTHON'))
job_data_analyst_final['tools_mentioned'] = job_data_analyst_final['tools_mentioned'].apply(lambda x: replace_word(x, 'POWERBI', 'POWER BI'))
job_data_analyst_final['tools_mentioned'] = job_data_analyst_final['tools_mentioned'].apply(lambda x: replace_word(x, 'BIG QUERY', 'BIGQUERY'))

# HARD SKILL DETECTION
job_data_analyst_final['hardskills_mentioned'] = job_data_analyst_final['job_description'].apply(lambda x: find_matches(x, pattern_hardskills, list_hardskills))

# EXPERIENCE REQ DETECTION
job_data_analyst_final['need_experience'] = job_data_analyst_final['job_description'].apply(contains_keywords)

# LANGUAGE DETECTION
job_data_analyst_final['language_mentioned'] = job_data_analyst_final['job_description'].apply(lambda x: find_matches(x, pattern_lang, list_language))

In [28]:
job_data_analyst_final['platform'] = 'Jobstreet'

In [29]:
job_data_analyst_final['data_scraped_at'] = today_string

In [30]:
job_data_analyst_final.to_excel(f'{today_string}_jobstreet_data_analyst.xlsx', index=False)

## Hasil crawling

In [31]:
import os

In [32]:
da_linkedin = glob.glob('*linkedin_data_analyst.xlsx')
da_jobstreet = glob.glob('*jobstreet_data_analyst.xlsx')

In [52]:
job_linkedin = pd.read_excel(da_linkedin[-1])

In [53]:
job_all_aggregate = pd.concat([job_data_analyst_final, job_linkedin], ignore_index=True)

In [54]:
# Get the current date
current_date = datetime.now()

# Function to calculate the actual post date
def calculate_post_date(days_ago):
    return current_date - timedelta(days=days_ago)

In [55]:
job_all_aggregate[job_all_aggregate['posted_since_days_ago'].isnull()]

,company,role,location,post_duration,total_application,job_description,level,employment_status,job_function,industries,...,company_total_employees,company_rating,company_total_review,posted_since_days_ago,data_scraped_at,tools_mentioned,hardskills_mentioned,need_experience,language_mentioned,platform


In [56]:
# Apply the function to the 'posted_since_days_ago' column
job_all_aggregate['post_date'] = job_all_aggregate['posted_since_days_ago'].apply(calculate_post_date)

# Format the date to 'YYYY-mm-dd'
job_all_aggregate['post_date'] = job_all_aggregate['post_date'].dt.strftime('%Y-%m-%d')

In [57]:
def extract_salary_range(salary_str):
    if pd.isna(salary_str):
        return np.nan, np.nan
    salary_str = salary_str.replace('Rp', '').replace('per month', '').replace('–', '-').replace('.', '').replace(',', '').replace('IDR', '')
    min_salary, max_salary = salary_str.split('-')
    return int(min_salary.strip()), int(max_salary.strip())

In [58]:
job_all_aggregate[['min_salary', 'max_salary']] = job_all_aggregate['salary'].apply(lambda x: pd.Series(extract_salary_range(x)))

In [59]:
master_job = job_all_aggregate[['job_id', 'company', 'role', 'post_date']]
master_job.to_excel('master_date_jobpost.xlsx', index=False)

In [60]:
# defining the scope of the application
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']

#credentials to the account
cred = ServiceAccountCredentials.from_json_keyfile_name('D:/7. Project/nibble-scraping-e8bfcb2c2f4a.json',scope)

# authorize the clientsheet
client = gspread.authorize(cred)

In [61]:
data_scrap = job_all_aggregate.values.tolist()

# Access an existing Google Sheet by title
spreadsheet = client.open('job_crawler_2024')

# Select the first worksheet of the spreadsheet
worksheet = spreadsheet.get_worksheet(0)

# Upload the data
worksheet.clear()  # Clear existing data (optional)
set_with_dataframe(worksheet, job_all_aggregate)